# Accident Detector

by Samuel E

This Python script requests accident and camera data in JSON format and returns location matches or approximates using distance calculations.

In [20]:
pip install geopy

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [21]:
import requests
from geopy.distance import geodesic
from urllib.parse import urlparse
from os.path import split
from io import FileIO

def download_incident_data(path):
    base_url = "https://travelmidwest.com/lmiga/incidents.json"
    params = {"path": path}

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        incidents_data = response.json()
        return incidents_data
    else:
        print(f"Error: Unable to fetch incident data. Status code: {response.status_code}")
        return None


def download_camera_data(state):
    base_url = "https://travelmidwest.com/lmiga/cameras.json"
    params = {"state": state, "idPrefix": ""}

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        camera_data = response.json()
        return camera_data
    else:
        print(f"Error: Unable to fetch camera data. Status code: {response.status_code}")
        return None


def find_nearest_camera(incident_lat, incident_lon, camera_list):
    nearest_camera = None
    min_distance = float('inf')

    for camera in camera_list:
        camera_lat = camera["geometry"]["coordinates"][1]
        camera_lon = camera["geometry"]["coordinates"][0]

        # need to figure out if this actually works
        distance = geodesic((incident_lat, incident_lon), (camera_lat, camera_lon)).kilometers

        if distance < min_distance:
            min_distance = distance
            nearest_camera = camera

    return nearest_camera, min_distance


def download_camera_images(camera):
    images = list()
    for url_and_direction in camera["properties"]["urls"]:
        direction = url_and_direction['direction']
        thumbnail = url_and_direction['thumbnail']
        url = url_and_direction['url']
        response = requests.get(url)
        if response.status_code == 200:
            image = response.content
            images.append({"direction": direction, "url": url, "image": image})
        else:
            print(f"Error: Unable to fetch camera image. Status code: {response.status_code}")
            return None
    return images


def save_camera_images(image_dicts):
    for image_dict in image_dicts:
        url = image_dict["url"]
        image = image_dict["image"]
        direction = image_dict["direction"]
        parsed_url = urlparse(url)
        filename = split(parsed_url.path)[1]
        with FileIO(filename, "wb") as f:
            f.write(image)
        print(filename + " saved")

def main():
    incident_path = "GATEWAY.IL"  # Replace with the incident path
    camera_state = "Illinois"  # Replace with the camera path

    incidents_data = download_incident_data(incident_path)
    camera_data = download_camera_data(camera_state)

    if incidents_data and camera_data:
        for incident_group in incidents_data:
            for incident_report in incident_group["reportRows"]:
                incident_lat = incident_report["latitude"]
                incident_lon = incident_report["longitude"]

                nearest_camera, distance = find_nearest_camera(incident_lat, incident_lon, camera_data["features"])

                if nearest_camera and distance < 2.0:
                    print(f"Incident Location: {incident_report['location']}")
                    print(f"Nearest Camera: {nearest_camera['properties']['id']}")
                    print(f"Camera Location: {nearest_camera['properties']['description']}")
                    print(
                        f"Distance to Camera: {distance:.2f} km")
                    print("-" * 30)
                    save_camera_images(download_camera_images(nearest_camera))
                else:
                    print(f"No cameras found near the incident at {incident_report['fullLocation']}")
                    print("-" * 30)


if __name__ == "__main__":
    main()


No cameras found near the incident at Ramp from SB I-90 to SB I-55, Chicago, Cook, IL
------------------------------
No cameras found near the incident at Ramp from SB I-90 to SB I-55, Chicago, Cook, IL
------------------------------
No cameras found near the incident at Ramp from SB I-90 to SB I-55, Chicago, Cook, IL
------------------------------
No cameras found near the incident at Ramp from SB I-90 to SB I-55, Chicago, Cook, IL
------------------------------
No cameras found near the incident at Ramp from SB I-90 to SB I-55, Chicago, Cook, IL
------------------------------
No cameras found near the incident at Ramp from SB I-90 to SB I-55, Chicago, Cook, IL
------------------------------
No cameras found near the incident at Ramp from SB I-90 to SB I-55, Chicago, Cook, IL
------------------------------
No cameras found near the incident at EB 26th St at California Ave, Chicago, Cook, IL
------------------------------
No cameras found near the incident at SEB US-14 at Greenwood Ave